##  Data Preprocessing

Step 2: 

Perform various transformations on our data as required for efficient and effective processing by Machine Learning algorithms based on the type and needs of individual features, identified through the data explorations performed in Step 1 - Data Exploration - Application Data.

### Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display # Allows the use of display() for DataFrames
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, LabelBinarizer, Imputer

# Show all of the data in a dataframe
pd.set_option('display.max_columns', None)

In [2]:
# Training Set
application_train = pd.read_csv('data/tmp/application_train_bureau_data_merged.csv', index_col=0)
application_test = pd.read_csv('data/tmp/application_test_bureau_data_merged.csv', index_col=0)
print("Loaded Training Set: {0} rows {1} cols".format(application_train.shape[0], application_train.shape[1]))
print("Loaded Testing Set: {0} rows {1} cols".format(application_test.shape[0], application_test.shape[1]))

Loaded Training Set: 307511 rows 320 cols
Loaded Testing Set: 48744 rows 319 cols


### Flags

In [3]:
use_imputer = 'Most_Frequent'      # Valid values are 'Mean', 'Median', 'Most_Frequent' and 'None'
use_replace_outliers = True        # Remove Outliers from Dataset
use_scaler = 'Standard'              # Valid values are 'Robust', 'Standard', 'MinMax", 'None'
use_log_transform = None

In [4]:
target_train = application_train['TARGET']
application_train_SK_ID_CURR = application_train['SK_ID_CURR']
application_train = application_train.drop('SK_ID_CURR', axis=1)
application_train = application_train.drop('TARGET', axis=1)

application_test_SK_ID_CURR = application_test['SK_ID_CURR']
application_test = application_test.drop('SK_ID_CURR', axis=1)

In [5]:
display(application_train.head())
display(application_test.head())

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_MOBIL,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUR_APPR_PROCESS_START_0,HOUR_APPR_PROCESS_START_1,HOUR_APPR_PROCESS_START_10,HOUR_APPR_PROCESS_START_11,HOUR_APPR_PROCESS_START_12,HOUR_APPR_PROCESS_START_13,HOUR_APPR_PROCESS_START_14,HOUR_APPR_PROCESS_START_15,HOUR_APPR_PROCESS_START_16,HOUR_APPR_PROCESS_START_17,HOUR_APPR_PROCESS_START_18,HOUR_APPR_PROCESS_START_19,HOUR_APPR_PROCESS_START_2,HOUR_APPR_PROCESS_START_20,HOUR_APPR_PROCESS_START_21,HOUR_APPR_PROCESS_START_22,HOUR_APPR_PROCESS_START_23,HOUR_APPR_PROCESS_START_3,HOUR_APPR_PROCESS_START_4,HOUR_APPR_PROCESS_START_5,HOUR_APPR_PROCESS_START_6,HOUR_APPR_PROCESS_START_7,HOUR_APPR_PROCESS_START_8,HOUR_APPR_PROCESS_START_9,HOUSETYPE_MODE_not specified,HOUSETYPE_MODE_org spec account,HOUSETYPE_MODE_reg oper account,HOUSETYPE_MODE_reg oper spec account,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUP

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_MOBIL,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUR_APPR_PROCESS_START_0,HOUR_APPR_PROCESS_START_1,HOUR_APPR_PROCESS_START_10,HOUR_APPR_PROCESS_START_11,HOUR_APPR_PROCESS_START_12,HOUR_APPR_PROCESS_START_13,HOUR_APPR_PROCESS_START_14,HOUR_APPR_PROCESS_START_15,HOUR_APPR_PROCESS_START_16,HOUR_APPR_PROCESS_START_17,HOUR_APPR_PROCESS_START_18,HOUR_APPR_PROCESS_START_19,HOUR_APPR_PROCESS_START_2,HOUR_APPR_PROCESS_START_20,HOUR_APPR_PROCESS_START_21,HOUR_APPR_PROCESS_START_22,HOUR_APPR_PROCESS_START_23,HOUR_APPR_PROCESS_START_3,HOUR_APPR_PROCESS_START_4,HOUR_APPR_PROCESS_START_5,HOUR_APPR_PROCESS_START_6,HOUR_APPR_PROCESS_START_7,HOUR_APPR_PROCESS_START_8,HOUR_APPR_PROCESS_START_9,HOUSETYPE_MODE_not specified,HOUSETYPE_MODE_org spec account,HOUSETYPE_MODE_reg oper account,HOUSETYPE_MODE_reg oper spec account,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUP

In [28]:
# Load the list of features indentified in the exploration step which need preprocessing
#non_numeric_features = pd.read_csv('data/tmp/bureau_non_numeric_features.csv', header=0, index_col=0, names=["feature"])
non_numeric_features = []
bureau_numeric_features = pd.read_csv('data/tmp/bureau_numeric_features.csv', header=0, index_col=0, names=["feature"])
application_numeric_features = pd.read_csv('data/tmp/numeric_features.csv', header=0, index_col=0, names=["feature"])
#string_to_bool_features = pd.read_csv('data/tmp/bureau_string_to_bool.csv', header=0, index_col=0, names=["feature"])
string_to_bool_features = []
log_transform_features = pd.read_csv('data/tmp/bureau_log_transform.csv', header=0, index_col=0, names=["feature"])

In [25]:
display(numeric_features.head)

<bound method NDFrame.head of                        feature
0           NUM_BUREAU_REPORTS
1          SUM_BUREAU_BALANCES
2   SUM_CREDIT_CURRENCY_CODE_0
3   SUM_CREDIT_CURRENCY_CODE_1
4   SUM_CREDIT_CURRENCY_CODE_2
5   SUM_CREDIT_CURRENCY_CODE_3
6     SUM_CREDIT_ACTIVE_CODE_0
7     SUM_CREDIT_ACTIVE_CODE_1
8     SUM_CREDIT_ACTIVE_CODE_2
9     SUM_CREDIT_ACTIVE_CODE_3
10      SUM_CREDIT_TYPE_CODE_0
11      SUM_CREDIT_TYPE_CODE_1
12      SUM_CREDIT_TYPE_CODE_2
13      SUM_CREDIT_TYPE_CODE_3
14      SUM_CREDIT_TYPE_CODE_4
15      SUM_CREDIT_TYPE_CODE_5
16      SUM_CREDIT_TYPE_CODE_6
17      SUM_CREDIT_TYPE_CODE_7
18      SUM_CREDIT_TYPE_CODE_8
19      SUM_CREDIT_TYPE_CODE_9
20     SUM_CREDIT_TYPE_CODE_10
21     SUM_CREDIT_TYPE_CODE_11
22     SUM_CREDIT_TYPE_CODE_12
23     SUM_CREDIT_TYPE_CODE_13
24     SUM_CREDIT_TYPE_CODE_14
25                SUM_STATUS_0
26                SUM_STATUS_1
27                SUM_STATUS_2
28                SUM_STATUS_3
29                SUM_STATUS_4
30       

In [8]:
print("Non-Numeric {0}, Numeric {1}, Bool {2}, Log Transform {3}".format(\
        len(non_numeric_features), len(numeric_features), \
        len(string_to_bool_features), len(log_transform_features)))

Non-Numeric 0, Numeric 42, Bool 0, Log Transform 1


In [9]:
''' 
Applies an in-place transformation that converts a Y/N field to binary 1/0
'''
def make_bool(dataFrame, featureName): 
    
    lb = LabelBinarizer()
    bool_name = "BOOL_{0}".format(featureName)    
    dataFrame[bool_name] = lb.fit_transform(dataFrame[featureName])
    
    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])    
    #result = dataFrame.drop(featureName, axis=1) 
    
    return dataFrame

In [10]:
''' 
Applies an in-place transformation that converts NaN, Inf and -Inf to numeric values
'''
def make_numeric(dataFrame, featureName): 
        
    return np.nan_to_num(dataFrame[featureName])

In [11]:
'''
Applies an in-place abs() to numeric features
'''
def abs_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: abs(x))
        
    dataFrame[featureName] = transformed
    
    return dataFrame

In [12]:
'''
Applies an in-place log transformation to numeric features
'''
def log_transform(dataFrame, featureName):     
    
    transformed = dataFrame[featureName].apply(lambda x: np.log(x + 1))
    
    transformed_name = "LOG_{0}".format(featureName)

    print("Transformed Name: {0}".format(transformed_name))
    
    dataFrame[transformed_name] = transformed

    # This seems to work more consistently than df.drop
    # From: https://stackoverflow.com/questions/43838198/df-drop-is-not-working
    del(dataFrame[featureName])
    #result = dataFrame.drop(featureName, axis=1)
    
    return dataFrame

In [13]:
'''
Applies an in-place transformation that offsets all values in a column by the minimum value in the column
'''

def find_offset(dataFrames, featureName): 
    
    offset = 0
    
    for df in dataFrames: 
        if (np.min(df[featureName]) < offset): 
            offset = np.min(df[featureName])

    return abs(offset) + 1
            
            
def offset_negative_values(dataFrame, featureName, offset): 
    print("Offsetting Feature {0} by {1}".format(featureName, offset))
    return dataFrame[featureName].apply(lambda x: (x + offset)), offset

In [14]:
# Tukey's Method for detecting outliers
# Adapted from the customer segments exercise

def return_outliers(dataFrame, featureList): 
    
    outliers = []
    count = 0
    
    frame_stats = []

    for feature_count, feature in enumerate(featureList): 

        feature_stats = {}

        dataFrame[feature] = np.nan_to_num(dataFrame[feature])

        # Calculate Q1 (25th percentile of the data) for the given feature
        Q1 = np.percentile(dataFrame[feature], 25)

        # Calculate Q3 (75th percentile of the data) for the given feature
        Q3 = np.percentile(dataFrame[feature], 75)

        # TODO: Use the interquartile range to calculate an outlier step
        step = (Q3 - Q1) * 1.5

        feature_stats['Q1_percentile'] = Q1
        feature_stats['Q3_percentile'] = Q3
        feature_stats['step'] = step
        feature_stats['feature'] = feature
        feature_stats['feature_min'] = np.min(dataFrame[feature])
        feature_stats['feature_max'] = np.max(dataFrame[feature])
        feature_stats['feature_median'] = np.median(dataFrame[feature])

        # Calculate the median of the interquartile range
        # TODO: This just sounds good intuitively.  It's probably worth researching the actual best practice.
        feature_stats['IQ_median'] = np.median([Q1, Q3])

        # Save a list of the outliers
        feature_stats['outliers'] = dataFrame.index[~((dataFrame[feature] >= Q1 - step) & (dataFrame[feature] <= Q3 + step))]

        # Just keep a count of the number of fields we identified
        count += len(feature_stats['outliers'])

        frame_stats.append(feature_stats)

    outliers.append(frame_stats)   
        
    print("Total Outliers Identified: {0}".format(count))
        
    return outliers

In [15]:
def replace_outliers(dataFrame, outliers):
    
    df = dataFrame.copy()
    
    count = 0
    
    for ol in outliers: 

        for feature_count, feature in enumerate(ol): 

            # Some features have infrequent but meaningful values, and averaging to 0 would be a problem.
            # We'll just skip these when they come up.
            # example: "How many credit applications were made in the past hour?" 
            if(feature['IQ_median'] >= 1): 

                featureName = feature['feature']
                print("Processing feature {0}, max: {1}, min {2}, mean {3}]".format(featureName, \
                        np.max(df[featureName]), np.min(df[featureName]), np.mean(df[featureName])))
            
                try: 
                    df.iloc[feature['outliers'], df.columns.get_loc(featureName)] = feature['IQ_median']
                    count += len(feature['outliers'])
                except Exception as e:
                        print("Error: {0}".format(str(e)))
 
                print("Processing Complete for {0}, max: {1}, min {2}, mean {3}]".format(featureName, \
                        np.max(df[featureName]), np.min(df[featureName]), np.mean(df[featureName])))
 

    print("Total Outliers Modified: {0}".format(count))
    return df

## Preprocessing

### Transform Date Fields to Positive Integers

In [16]:
application_train = abs_transform(application_train, 'SUM_DAYS_ENDDATE_FACT')
application_test = abs_transform(application_test, 'SUM_DAYS_ENDDATE_FACT')

application_train = abs_transform(application_train, 'SUM_DAYS_CREDIT_ENDDATE')
application_test = abs_transform(application_test, 'SUM_DAYS_CREDIT_ENDDATE')

application_train = abs_transform(application_train, 'SUM_DAYS_CREDIT_UPDATE')
application_test = abs_transform(application_test, 'SUM_DAYS_CREDIT_UPDATE')

In [17]:
display(application_train.head())

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_MOBIL,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUR_APPR_PROCESS_START_0,HOUR_APPR_PROCESS_START_1,HOUR_APPR_PROCESS_START_10,HOUR_APPR_PROCESS_START_11,HOUR_APPR_PROCESS_START_12,HOUR_APPR_PROCESS_START_13,HOUR_APPR_PROCESS_START_14,HOUR_APPR_PROCESS_START_15,HOUR_APPR_PROCESS_START_16,HOUR_APPR_PROCESS_START_17,HOUR_APPR_PROCESS_START_18,HOUR_APPR_PROCESS_START_19,HOUR_APPR_PROCESS_START_2,HOUR_APPR_PROCESS_START_20,HOUR_APPR_PROCESS_START_21,HOUR_APPR_PROCESS_START_22,HOUR_APPR_PROCESS_START_23,HOUR_APPR_PROCESS_START_3,HOUR_APPR_PROCESS_START_4,HOUR_APPR_PROCESS_START_5,HOUR_APPR_PROCESS_START_6,HOUR_APPR_PROCESS_START_7,HOUR_APPR_PROCESS_START_8,HOUR_APPR_PROCESS_START_9,HOUSETYPE_MODE_not specified,HOUSETYPE_MODE_org spec account,HOUSETYPE_MODE_reg oper account,HOUSETYPE_MODE_reg oper spec account,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUP

### Remove Outliers

In [18]:
numeric = [] 

for feature in log_transform_features['feature']:
    numeric.append("{0}".format(feature))

for feature in numeric_features['feature']: 
    numeric.append("{0}".format(feature))

if use_replace_outliers is True: 
    print("Replacing Outliers")
    print("Before: AMT_INCOME_TOTAL Max: {0}".format(np.max(application_train['AMT_INCOME_TOTAL'])))
    outliers_train = return_outliers(application_train, numeric)
    outliers_test = return_outliers(application_test, numeric)
    application_train = replace_outliers(application_train, outliers_train)
    application_test = replace_outliers(application_test, outliers_test)
    print("After: AMT_INCOME_TOTAL Max: {0}".format(np.max(application_train['AMT_INCOME_TOTAL'])))

Replacing Outliers
Before: AMT_INCOME_TOTAL Max: 2.9275127246996657
Total Outliers Identified: 1045936
Total Outliers Identified: 0
Processing feature NUM_BUREAU_REPORTS, max: 116.0, min 0.0, mean 1.7034805258998864]
Processing Complete for NUM_BUREAU_REPORTS, max: 5.0, min 0.0, mean 0.6243516492092965]
Processing feature SUM_BUREAU_BALANCES, max: 2657.0, min 0.0, mean 47.80841010565476]
Processing Complete for SUM_BUREAU_BALANCES, max: 72.0, min 0.0, mean 6.492499455304038]
Processing feature SUM_CREDIT_CURRENCY_CODE_0, max: 116.0, min 0.0, mean 1.7012204441467134]
Processing Complete for SUM_CREDIT_CURRENCY_CODE_0, max: 5.0, min 0.0, mean 0.6244784739407696]
Processing feature SUM_STATUS_0, max: 696.0, min 0.0, mean 15.009817534982488]
Processing Complete for SUM_STATUS_0, max: 22.0, min 0.0, mean 2.0333532784193085]
Processing feature SUM_DAYS_CREDIT_ENDDATE, max: 214193.0, min 0.0, mean 2098.2946105992955]
Processing Complete for SUM_DAYS_CREDIT_ENDDATE, max: 1370.0, min 0.0, mean 

### Remove Non-Numeric Values from Numeric Fields

In [20]:
# Assemble a list of all numeric fields
numeric = [] 

for feature in log_transform_features['feature']:
    numeric.append("{0}".format(feature))

for feature in numeric_features['feature']: 
    numeric.append("{0}".format(feature))

if str.lower(use_imputer) != 'none': 

    print("Using Imputer {0}".format(use_imputer))
    
    imputer = Imputer(strategy=str.lower(use_imputer))
    application_train[numeric] = imputer.fit_transform(application_train[numeric])
    application_test[numeric] = imputer.fit_transform(application_test[numeric])

else: 
    
    print("Using np.nan_to_num()")
    numeric = list(application_test.select_dtypes(include=[np.number]).columns.values)

    for feature in numeric: 
        application_train[feature] = make_numeric(application_train, feature)
        application_test[feature] = make_numeric(application_test, feature)

Using Imputer Most_Frequent


### Log Transform Skewed Numeric Features

In [19]:
''' 
# Skipping Log transformations for the time being... 

# Ensure all values are numeric, then log transform them
for feature in log_transform_features['feature']: 
        application_train[feature] = make_numeric(application_train, feature)
    
        print("Performing Log Transformations on: {0}".format(feature))
        application_train = log_transform(application_train, feature)
'''

' \n# Skipping Log transformations for the time being... \n\n# Ensure all values are numeric, then log transform them\nfor feature in log_transform_features[\'feature\']: \n        application_train[feature] = make_numeric(application_train, feature)\n    \n        print("Performing Log Transformations on: {0}".format(feature))\n        application_train = log_transform(application_train, feature)\n'

### Scale Numeric Fields

In [29]:
# Initialize a scaler, then apply it to the features

if str.lower(use_scaler) == 'robust': 
    scaler = RobustScaler()

if str.lower(use_scaler) == 'standard': 
    scaler = StandardScaler()
    
if str.lower(use_scaler) == 'minmax': 
    scaler = MinMaxScaler()
    
if str.lower(use_scaler) != 'none':

    # Assemble a list of all numeric fields
    numeric = [] 

    if use_log_transform is True: 
    
        for feature in log_transform_features['feature']:
            numeric.append("LOG_{0}".format(feature))

    else: 

        for feature in log_transform_features['feature']:
            numeric.append("{0}".format(feature))
        
            
    for feature in numeric_features['feature']: 
        numeric.append("{0}".format(feature))
        
    for feature in application_numeric_features['feature']:
        numeric.append("{0}".format(feature))
        
    # Technique for selecting numeric fields from: 
    # https://stackoverflow.com/questions/25039626/how-do-i-find-numeric-columns-in-pandas
    # numeric = list(application_test.select_dtypes(include=[np.number]).columns.values)

    scaler = scaler.fit(application_train[numeric])

print("Using Scaler: {0}".format(use_scaler))

Using Scaler: Standard


In [30]:
application_train[numeric] = scaler.transform(application_train[numeric])
application_test[numeric] = scaler.transform(application_test[numeric])

### Preview
Show our transformed dataset

In [31]:
# Show an example of a record with scaling applied
display(application_train.head(n = 5))

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,APARTMENTS_AVG,APARTMENTS_MEDI,APARTMENTS_MODE,BASEMENTAREA_AVG,BASEMENTAREA_MEDI,BASEMENTAREA_MODE,BOOL_FLAG_EMAIL,BOOL_FLAG_EMP_PHONE,BOOL_FLAG_OWN_CAR,BOOL_FLAG_OWN_REALTY,BOOL_FLAG_PHONE,BOOL_FLAG_WORK_PHONE,CNT_CHILDREN,CNT_FAM_MEMBERS,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,COMMONAREA_AVG,COMMONAREA_MEDI,COMMONAREA_MODE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_ID_PUBLISH,DAYS_LAST_PHONE_CHANGE,DAYS_REGISTRATION,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,ELEVATORS_AVG,ELEVATORS_MEDI,ELEVATORS_MODE,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,ENTRANCES_AVG,ENTRANCES_MEDI,ENTRANCES_MODE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,FLAG_CONT_MOBILE,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_2,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_MOBIL,FLOORSMAX_AVG,FLOORSMAX_MEDI,FLOORSMAX_MODE,FLOORSMIN_AVG,FLOORSMIN_MEDI,FLOORSMIN_MODE,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUR_APPR_PROCESS_START_0,HOUR_APPR_PROCESS_START_1,HOUR_APPR_PROCESS_START_10,HOUR_APPR_PROCESS_START_11,HOUR_APPR_PROCESS_START_12,HOUR_APPR_PROCESS_START_13,HOUR_APPR_PROCESS_START_14,HOUR_APPR_PROCESS_START_15,HOUR_APPR_PROCESS_START_16,HOUR_APPR_PROCESS_START_17,HOUR_APPR_PROCESS_START_18,HOUR_APPR_PROCESS_START_19,HOUR_APPR_PROCESS_START_2,HOUR_APPR_PROCESS_START_20,HOUR_APPR_PROCESS_START_21,HOUR_APPR_PROCESS_START_22,HOUR_APPR_PROCESS_START_23,HOUR_APPR_PROCESS_START_3,HOUR_APPR_PROCESS_START_4,HOUR_APPR_PROCESS_START_5,HOUR_APPR_PROCESS_START_6,HOUR_APPR_PROCESS_START_7,HOUR_APPR_PROCESS_START_8,HOUR_APPR_PROCESS_START_9,HOUSETYPE_MODE_not specified,HOUSETYPE_MODE_org spec account,HOUSETYPE_MODE_reg oper account,HOUSETYPE_MODE_reg oper spec account,LANDAREA_AVG,LANDAREA_MEDI,LANDAREA_MODE,LIVE_CITY_NOT_WORK_CITY,LIVE_REGION_NOT_WORK_REGION,LIVINGAPARTMENTS_AVG,LIVINGAPARTMENTS_MEDI,LIVINGAPARTMENTS_MODE,LIVINGAREA_AVG,LIVINGAREA_MEDI,LIVINGAREA_MODE,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NONLIVINGAPARTMENTS_AVG,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_AVG,NONLIVINGAREA_MEDI,NONLIVINGAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUP

### Output
Save out our preprocessed data to temporary intermediate files

In [32]:
application_train['SK_ID_CURR'] = application_train_SK_ID_CURR
application_train['TARGET'] = target_train
application_test['SK_ID_CURR'] = application_test_SK_ID_CURR

train_filename = 'data/tmp/bureau_train_preprocessed_'
test_filename = 'data/tmp/bureau_test_preprocessed_'

if str.lower(use_imputer) is not 'none': 
    train_filename += "imputation_{0}_".format(str.lower(use_imputer))
    test_filename += "imputation_{0}_".format(str.lower(use_imputer))

if use_scaler is 'None':
    train_filename += "no_scaling_"
    test_filename += "no_scaling_"
else: 
    train_filename += "{0}_scaler_".format(str.lower(use_scaler))
    test_filename += "{0}_scaler_".format(str.lower(use_scaler))

if use_replace_outliers is True: 
    train_filename += "outliers_replaced_"
    test_filename += "outliers_replaced_"
else: 
    train_filename += "with_outliers_"
    test_filename += "with_outliers_"

if use_log_transform is True: 
    train_filename += "log_transform_monetary_"
    test_filename += "log_transform_monetary_"
else: 
    train_filename += "no_log"
    test_filename += "no_log"
    
train_filename += ".csv"
test_filename += ".csv"

print("Writing {0}".format(train_filename))
application_train.to_csv(train_filename)
print("Writing {0}".format(test_filename))
application_test.to_csv(test_filename)
print("Output Complete")

Writing data/tmp/bureau_train_preprocessed_imputation_most_frequent_standard_scaler_outliers_replaced_no_log.csv
Writing data/tmp/bureau_test_preprocessed_imputation_most_frequent_standard_scaler_outliers_replaced_no_log.csv
Output Complete
